In [1]:
import numpy as np
import pandas as pd

In [2]:
TRAIN_PATH = "archive/data/data/train/A-3.npy"
TEST_PATH  = "archive/data/data/test/A-3.npy"


In [ ]:
X_train = np.load(TRAIN_PATH)
X_test  = np.load(TEST_PATH)

X_train = X_train.reshape(-1, 1)
X_test  = X_test.reshape(-1, 1)

print("Train shape:", X_train.shape)
print("Test shape :", X_test.shape)


Train shape: (68400, 1)
Test shape : (205125, 1)


In [ ]:
from sklearn.ensemble import IsolationForest

model = IsolationForest(
    n_estimators=100,
    contamination=0.05,   
    random_state=42
)

model.fit(X_train)

print("Model trained on normal behavior.")


Model trained on normal behavior.


In [7]:
predictions = model.predict(X_test)

anomaly_labels = np.where(predictions == -1, 1, 0)

print("Total test points:", len(anomaly_labels))
print("Detected anomalies:", anomaly_labels.sum())


Total test points: 205125
Detected anomalies: 5146
